# EEC 174AY 2023 Fall Mini Project B1: Improve your ML model

## Introduction

This asignment follows Lab2 and uses the same dataset as Lab2.

### Reading Data into Memory

In [6]:
# make sure we can plot in future if we want
%matplotlib notebook
# make sure to ignore warnings
import warnings
warnings.simplefilter('ignore')
# Import statement for pandas
import pandas as pd
# This is just a small configuration change for purposes of the class
pd.options.display.max_rows = 10

# Get our train X and y datasets for the problem
train_x = pd.read_csv('ece174_pva_train_x.csv')
train_y = pd.read_csv('ece174_pva_train_y.csv')

# Get our validation X and y datasets for the problem.
test_x = pd.read_csv('ece174_pva_validation_x.csv')
test_y = pd.read_csv('ece174_pva_validation_y.csv')

In [7]:
# output some rows of the dataset just to get a better feel for the information
train_x

,breath_id,i_time,tve,max_flow,min_flow,max_pressure,peep,ip_auc,ep_auc,patient
0,1,0.80,545.032222,51.06,-41.03,17.37,7.600,11.122367,16.057733,66
1,2,0.80,531.880278,53.13,-39.97,17.13,7.508,11.077750,17.310533,66
2,3,0.86,523.876667,52.86,-38.24,17.11,7.658,12.066000,16.697800,66
3,4,0.80,507.636111,51.04,-39.37,17.14,7.572,11.097800,15.774250,66
4,5,0.80,518.618889,47.88,-38.51,16.92,7.598,11.065400,18.483333,66
...,...,...,...,...,...,...,...,...,...,...
5970,296,0.90,355.365278,42.26,-51.51,23.53,13.194,19.216400,21.816367,662
5971,297,0.90,316.806944,42.10,-55.17,24.61,12.896,19.800467,21.739700,662
5972,298,0.92,395.971111,42.95,-22.47,21.35,13.090,16.997767,21.457600,662
5973,299,0.90,373.426389,40.34,-36.81,21.69,13.334,17.944000,21.798167,662


Here we have 18 columns. I'm going to give a detailed breakdown here. Feel free to come back to it as necessary.

Features:
 * *breath_id* - matches with a specific breath identifier from the raw data file.
 * *patient* - the patient the data came from
 * *min_flow* - The minimum flow observation on the breath
 * *max_flow* - The maximum flow observation on the breath
 * *tvi* - The inhaled volume of air for each breath
 * *tve* - The exhaled volume of air for each breath
 * *tve_tvi_ratio* - The ratio of `tve / tvi`
 * *i_time* - The amount of time patient was breathing in for each breath
 * *e_time* - The amount of time patient was breathing out for each breath
 * *ie_ratio* - The ratio of `i_time / e_time`
 * *rr* - The respiratory rate in number of breaths per minute. Measured by `60 / (i_time + e_time)`
 * *min_pressure* - the minimum pressure observation on the breath
 * *max_pressure* - the maximum pressure observation on the breath
 * *peep* - the baseline pressure setting on the ventilator
 * *pip* - the maximum pressure setting of inspiration. Slight difference compared to max_pressure
 * *maw* - the mean pressure for the entire breath
 * *ip_auc* - the area under the curve of the inspiratory pressure
 * *ep_auc* - the area under the curve of the expiratory pressure

## Featurization

Featurization is the process where you extract information from raw data. This information can then be fed into a machine learning algorithm to perform the task you want. In the current case we will need to extract additional information from the ventilator data in order to create a valid machine learning classifier.

### Processing the Data
The first step we need to do is to be able to read the raw data files and put them into memory. We have taken this problem away from you for the purposes of this homework and have given you the code so that you can do this

In [8]:
import csv


def process_ventilator_data(filename):
    descriptor = open(filename)
    reader = csv.reader(descriptor)
    breath_id = 1

    all_breath_data = []
    current_flow_data = []
    current_pressure_data = []

    for row in reader:
        if (row[0].strip() == 'BS' or row[0].strip() == 'BE') and current_flow_data != []:
            all_breath_data.append({'breath_id': breath_id, 'flow': current_flow_data, 'pressure': current_pressure_data})
            breath_id += 1
            current_flow_data = []
            current_pressure_data = []
        else:
            try:
                current_flow_data.append(round(float(row[0]), 2))
                current_pressure_data.append(round(float(row[1]), 2))
            except (IndexError, ValueError):
                continue
    return all_breath_data

In [9]:
from glob import glob
import os

# import for Simpson's method. This will be helpful for calculating TVi
from scipy.integrate import simps
from statistics import mean


def extract_features_for_file(filename, existing_features):
    """
    Extract features for every single breath in file. To make matters a bit easier, we use
    existing features that we've already extracted from the file to help speed the process.
    """
    patient = filename.split('\\')[-2]
    all_breath_data = process_ventilator_data(filename)
    all_features = []

    for breath_data in all_breath_data:
        breath_id = breath_data['breath_id']
        existing_breath_features = existing_features[existing_features.breath_id == breath_id].iloc[0]

        flow = breath_data['flow']
        pressure = breath_data['pressure']

        # inspiratory time (the amount of time a patient is inhaling for)
        i_time = existing_breath_features.i_time
        # exhaled tidal volume
        tve = existing_breath_features.tve
        # maximum flow for breath
        max_flow = existing_breath_features.max_flow
        # minimum flow for the breath
        min_flow = existing_breath_features.min_flow
        # maximum pressure for the breath
        max_pressure = existing_breath_features.max_pressure
        # The minimum pressure setting on the ventilator
        peep = existing_breath_features.peep
        # The area under the curve of the inspiratory pressure curve
        ip_auc = existing_breath_features.ip_auc
        # The area under the curve of the expiratory pressure curve
        ep_auc = existing_breath_features.ep_auc

        # This is the array index where the inhalation ends. We divide by 0.02 because
        # thats how frequently the ventilator samples data, every 0.02 seconds.
        x0_index = int(i_time / 0.02)

        # Part of your assignment is to extract the following features for all breaths:
        #
        # Expiratory Time. The amount of time a patient is exhaling
        e_time = len(flow) * .02 - i_time
        #
        # I:E ratio. The ratio of inspiratory to expiratory time. Measured by i_time/e_time
        i_e_ratio = i_time / e_time
        #
        # Respiratory rate. The number of breaths a patient is breathing. This is measured by
        # 60 / (total breath time in seconds)
        rr = 60 / (i_time + e_time)
        rr = 60 / (len(flow) * .02)
        #
        # Tidal volume inhaled. The amount of air volume inhaled in the breath.
        # Hint: use the simps function.
        # This will output volume in L/min, convert to ml/sec (* 1000 / 60)
        tvi = simps(flow[:x0_index], dx=0.02) * 1000 / 60
        #
        # Tidal volume ratio. Measured by tve/tvi
        tve_tvi_ratio = tve / tvi
        #
        # Minimum pressure of the breath
        min_pressure = min(pressure)
        #
        # PIP - peak inspiratory pressure. The peak pressure during inhalation
        pip = max(pressure[:x0_index])
        #
        # MAW - mean airway pressure for inhalation.
        maw = mean(pressure[:x0_index])

        all_features.append([
            breath_id, i_time, e_time, i_e_ratio, rr, tvi, tve, tve_tvi_ratio,
            max_flow, min_flow, max_pressure, min_pressure, pip, maw,
            peep, ip_auc, ep_auc, int(patient)
        ])
    columns = [
        'breath_id', 'i_time', 'e_time', 'i_e_ratio', 'rr', 'tvi', 'tve',
        'tve_tvi_ratio', 'max_flow', 'min_flow', 'max_pressure',
        'min_pressure', 'pip', 'maw', 'peep', 'ip_auc', 'ep_auc', 'patient'
    ]
    return all_features, columns


def remake_dataset(dataset):
    data_files = glob(os.path.join('data', '*/*.csv'))

    patient_to_file_map = {}
    for filename in data_files:
        patient = filename.split('\\')[-2]  # patient is embedded in this part of filename
        patient_to_file_map[patient] = filename

    data = []
    # iterate over all the unique patients in the train set
    for patient in dataset.patient.unique():
        existing_features = dataset[dataset.patient == patient]
        filename = patient_to_file_map[str(patient)]
        breath_data, columns = extract_features_for_file(filename, existing_features)
        # add breath rows
        data.extend(breath_data)
    # create new data frame with the new added information
    return pd.DataFrame(data, columns=columns)

In [10]:
# remake train set
train_x = remake_dataset(train_x)
# remake validation set.
test_x = remake_dataset(test_x)

In [11]:
train_x

,breath_id,i_time,e_time,i_e_ratio,rr,tvi,tve,tve_tvi_ratio,max_flow,min_flow,max_pressure,min_pressure,pip,maw,peep,ip_auc,ep_auc,patient
0,1,0.80,1.66,0.481928,24.390244,481.917778,545.032222,1.130965,51.06,-41.03,17.37,7.04,17.37,14.208500,7.600,11.122367,16.057733,66
1,2,0.80,1.80,0.444444,23.076923,484.920278,531.880278,1.096841,53.13,-39.97,17.13,7.04,17.13,14.149500,7.508,11.077750,17.310533,66
2,3,0.86,1.74,0.494253,23.076923,521.370000,523.876667,1.004808,52.86,-38.24,17.11,7.04,17.11,14.311860,7.658,12.066000,16.697800,66
3,4,0.80,1.64,0.487805,24.590164,482.908333,507.636111,1.051206,51.04,-39.37,17.14,7.04,17.14,14.174500,7.572,11.097800,15.774250,66
4,5,0.80,1.94,0.412371,21.897810,466.349722,518.618889,1.112081,47.88,-38.51,16.92,7.04,16.92,14.131250,7.598,11.065400,18.483333,66
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
5970,296,0.90,1.60,0.562500,24.000000,349.682222,355.365278,1.016252,42.26,-51.51,23.53,13.00,23.53,21.750667,13.194,19.216400,21.816367,662
5971,297,0.90,1.60,0.562500,24.000000,350.130000,316.806944,0.904827,42.10,-55.17,24.61,12.80,24.61,22.414667,12.896,19.800467,21.739700,662
5972,298,0.92,1.58,0.582278,24.000000,355.670278,395.971111,1.113310,42.95,-22.47,21.35,12.90,21.35,18.785435,13.090,16.997767,21.457600,662
5973,299,0.90,1.60,0.562500,24.000000,337.941111,373.426389,1.105004,40.34,-36.81,21.69,13.02,21.69,20.308444,13.334,17.944000,21.798167,662


### Create Ground Truth (that the machine understands)

In [12]:
# Read the test dataset and set it up. Technically we're using the validation set.
test_y

,breath_id,patient,bsa,dta,cough,suction
0,20,292,1,0,0,0
1,21,292,1,0,0,0
2,22,292,0,0,0,0
3,23,292,1,0,0,0
4,24,292,1,0,0,0
...,...,...,...,...,...,...
1242,295,114,0,0,0,0
1243,296,114,0,0,0,0
1244,297,114,0,0,0,0
1245,298,114,0,0,0,0


What does this mean?

We have 6 columns here
 * *breath_id* - matches with a specific breath identifier from the raw data file.
 * *patient* - the patient the data came from
 * *bsa* - Breath Stacking Asynchrony. A single breath where the patient is trapping air in their chest
 * *dta* - Double Trigger Asynchrony. Two breaths in a row where the patient is trapping air
 * *cough* - What it sounds like, when a patient coughs
 * *suction* -  Nurses perform suction procedures to remove excess fluid from an endotracheal tube. This waveform is indicative of that.

Now that we understand what our columns are, we need to put it into a format where the machine can understand it and create a learning model. Because this is a multiclass model, let's just have non-PVA breaths be class 0, breath stacking can be class 1, double trigger can be class 2.

In [13]:
# Create a multi-class y vector that we can use for training purposes.
train_y_vector = train_y.bsa * 1 + train_y.dta * 2
test_y_vector = test_y.bsa * 1 + test_y.dta * 2
test_y_vector

0       1
1       1
2       0
3       1
4       1
       ..
1242    0
1243    0
1244    0
1245    0
1246    0
Length: 1247, dtype: int64

In [14]:
# See if there places where the data was mis-annotated, where both double trigger and breath stack was annotated.
# It's just good to know if this is happening or not so that we can either drop the data, or change it later on.
train_y_vector[train_y_vector > 2]

5438    3
5440    3
5521    3
dtype: int64

### Creating a Model


In [15]:
# Need to finalize dataset and remove misannotated examples first.

# just drop places where data is double annotated.
misannotated_train = train_y_vector > 2
misannotated_test = test_y_vector > 2

# ~ is the NOT operator
train_x = train_x.loc[~misannotated_train]
train_y_vector = train_y_vector.loc[~misannotated_train]

# do same thing for test
test_x = test_x.loc[~misannotated_test]
test_y_vector = test_y_vector.loc[~misannotated_test]



# Also make sure to drop data that is NaN. This is very important because otherwise your model won't train.
# The .any(axis=1) function basically says, if there are any nans in this *ROW* then mark the row as true.
# The .any(axis=0) would mark columns as True/False, but this isn't helpful now.
nans_train = train_x.isna().any(axis=1)
nans_test = test_x.isna().any(axis=1)

# now filter them out of the dataset in the same way
train_x = train_x.loc[~nans_train]
train_y_vector = train_y_vector.loc[~nans_train]

test_x = test_x.loc[~nans_test]
test_y_vector = test_y_vector.loc[~nans_test]


# any time we drop things from a data frame or series in pandas it is often helpful to re-index the object.
# the index is usually a sequential ordering of the rows like 1, 2, ... n. Sometimes it can be different
# but for now we'll just use sequential ordering
train_x.index = range(len(train_x))
train_y_vector.index = range(len(train_y_vector))

test_x.index = range(len(test_x))
test_y_vector.index = range(len(test_y_vector))

In [16]:
train_x

,breath_id,i_time,e_time,i_e_ratio,rr,tvi,tve,tve_tvi_ratio,max_flow,min_flow,max_pressure,min_pressure,pip,maw,peep,ip_auc,ep_auc,patient
0,1,0.80,1.66,0.481928,24.390244,481.917778,545.032222,1.130965,51.06,-41.03,17.37,7.04,17.37,14.208500,7.600,11.122367,16.057733,66
1,2,0.80,1.80,0.444444,23.076923,484.920278,531.880278,1.096841,53.13,-39.97,17.13,7.04,17.13,14.149500,7.508,11.077750,17.310533,66
2,3,0.86,1.74,0.494253,23.076923,521.370000,523.876667,1.004808,52.86,-38.24,17.11,7.04,17.11,14.311860,7.658,12.066000,16.697800,66
3,4,0.80,1.64,0.487805,24.590164,482.908333,507.636111,1.051206,51.04,-39.37,17.14,7.04,17.14,14.174500,7.572,11.097800,15.774250,66
4,5,0.80,1.94,0.412371,21.897810,466.349722,518.618889,1.112081,47.88,-38.51,16.92,7.04,16.92,14.131250,7.598,11.065400,18.483333,66
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
5967,296,0.90,1.60,0.562500,24.000000,349.682222,355.365278,1.016252,42.26,-51.51,23.53,13.00,23.53,21.750667,13.194,19.216400,21.816367,662
5968,297,0.90,1.60,0.562500,24.000000,350.130000,316.806944,0.904827,42.10,-55.17,24.61,12.80,24.61,22.414667,12.896,19.800467,21.739700,662
5969,298,0.92,1.58,0.582278,24.000000,355.670278,395.971111,1.113310,42.95,-22.47,21.35,12.90,21.35,18.785435,13.090,16.997767,21.457600,662
5970,299,0.90,1.60,0.562500,24.000000,337.941111,373.426389,1.105004,40.34,-36.81,21.69,13.02,21.69,20.308444,13.334,17.944000,21.798167,662


__From Lab2, we know there are only 3 classes in our labels as shown below.__

In [17]:
test_y

,breath_id,patient,bsa,dta,cough,suction
0,20,292,1,0,0,0
1,21,292,1,0,0,0
2,22,292,0,0,0,0
3,23,292,1,0,0,0
4,24,292,1,0,0,0
...,...,...,...,...,...,...
1242,295,114,0,0,0,0
1243,296,114,0,0,0,0
1244,297,114,0,0,0,0
1245,298,114,0,0,0,0


__What does this mean?__

We have 6 columns here
 * *breath_id* - matches with a specific breath identifier from the raw data file.
 * *patient* - the patient the data came from
 * *bsa* - Breath Stacking Asynchrony. A single breath where the patient is trapping air in their chest
 * *dta* - Double Trigger Asynchrony. Two breaths in a row where the patient is trapping air
 * *cough* - What it sounds like, when a patient coughs
 * *suction* -  Nurses perform suction procedures to remove excess fluid from an endotracheal tube. This waveform is indicative of that.

Now that we understand what our columns are, we need to put it into a format where the machine can understand it and create a learning model. Because this is a multiclass model, let's just have __non-PVA breaths be class 0, breath stacking can be class 1, double trigger can be class 2__.




```
# This is formatted as code
```

## Assignment \#1 Feature Selection

One thing to note is that we are using 16 different features for input into our model. Some of these features can be of little value to classifying whether a breath is asynchronous or not. So, one of the easiest things we can do for ourselves is to reduce the number of features that we have in an intelligent way.

### $\chi^2$ Feature Selection (chi squared)

Probably one of the easiest methods and intuitive methods to use for feature selection in classification problems. The [$\chi^2$ test](https://en.wikipedia.org/wiki/Chi-squared_test) measures whether a two statistical distributions are independent. In t[he applied case](https://nlp.stanford.edu/IR-book/html/htmledition/feature-selectionchi2-feature-selection-1.html), this means asking the question of whether a single feature is independent of the target vector. If a feature and the outcome are independent then this variable might not be helpful for our model. If a feature is independent of the outcome it will have a high chi2 value and a high pvalue. On the other hand, if a feature is not independent of the outcome, then it will have a high chi2 value and a low p-value (within range of 0-.05).

There is a function in scikit-learn that enables you to do the $\chi^2$ test.

In [18]:
# this is the PrettyPrint function. Just makes things look a bit nicer on output.
from pprint import pprint

from sklearn.feature_selection import chi2
from sklearn.preprocessing import MinMaxScaler

# get all columns in our dataset except patient and breath_id
columns_to_use = list(set(train_x.columns).difference(['patient', 'breath_id']))

# must scale feature vectors so they are non-negative
scaler = MinMaxScaler()
train_set = scaler.fit_transform(train_x[columns_to_use])

# the chi2 test will output two things, chi2 and p values. The p values are the most relevant item that we want
# to use. A feature with a p-value between 0 and 0.05 means that a feature might be a good predictor of our outcome.
chi2_vals, pvals = chi2(train_set, train_y_vector)

# mash column names with p-values so we know which p-value belongs to which feature
cols_to_pvals = zip(pvals, columns_to_use)
# Sort the p-values in ascending order (smallest first).
cols_sorted = sorted(cols_to_pvals)
# pretty print the sorted values.
pprint(cols_sorted)

[(2.084595661078224e-15, 'ep_auc'),
 (4.196362609447806e-10, 'tve'),
 (8.032273135900752e-06, 'e_time'),
 (9.867120183950849e-06, 'i_e_ratio'),
 (0.006478996818996394, 'rr'),
 (0.01875342252603949, 'i_time'),
 (0.05675212148035325, 'ip_auc'),
 (0.15346609771406564, 'max_pressure'),
 (0.41768345448170485, 'min_flow'),
 (0.4539978193632235, 'peep'),
 (0.5040246270515399, 'pip'),
 (0.6017652297104535, 'maw'),
 (0.6801265398980325, 'max_flow'),
 (0.8454199553593232, 'tvi'),
 (0.9029585211869112, 'min_pressure'),
 (0.9992552606338507, 'tve_tvi_ratio')]


There are 2 features that had p-values below 0.05:

 * tve
 * ep_auc

So let's use these features for our next model.

In [19]:
from sklearn.ensemble import RandomForestClassifier
from sklearn.preprocessing import MinMaxScaler
from sklearn.metrics import classification_report

model = RandomForestClassifier()
columns_to_use = ['tve', 'ep_auc']

scaler = MinMaxScaler()
train_set = scaler.fit_transform(train_x[columns_to_use])
test_set = scaler.transform(test_x[columns_to_use])

model.fit(train_set, train_y_vector)
predictions = model.predict(test_set)
for idx, pred in enumerate(predictions):
    if pred == 2:
        predictions[idx-1] = 2

print(classification_report(test_y_vector, predictions))

              precision    recall  f1-score   support

           0       0.76      0.77      0.77       842
           1       0.38      0.48      0.43       301
           2       0.13      0.02      0.03       104

    accuracy                           0.64      1247
   macro avg       0.43      0.42      0.41      1247
weighted avg       0.62      0.64      0.62      1247



Our performance actually dropped when we were trying to use $\chi^2$ test. Does this mean that the $\chi^2$ method isn't good for our problem?

What is happening above?

Even though the $\chi^2$ test is telling us these features are relevant to prediction, this just isn't the case in  the test set. This can happen frequently in machine learning, where information that is relevant to the training set doesn't generalize to the testing set. Are there other methods of feature selection which are more likely to generalize to the testing set?

### Expert Feature Selection

It always helps to have expert knowledge on the problem to improve model performance. In this case expert knowledge can be considered medical knowledge. So what kind of medical knowledge can we use to help this?

#### Breath Stack (BSA)
Remember the waveforms here? This means that the patient is trapping air in their chest. We can measure this via the `tve_tvi_ratio`. The way that our doctors annotated breaths was if the breaths had a `tve_tvi_ratio < .9` and they weren't a suction/cough or another anomaly.

<img src="bsa-breath.png" alt="Drawing" style="width: 400px;"/>

#### Double Trigger (DTA)
Double trigger has a double-hump pattern to it.

<img src="dta-breaths.png" alt="Drawing" style="width: 600px;"/>

The way our doctors annotated it was if

1. It wasn't an anomaly
2. First breath in sequence had an `e_time < .3` seconds
3. First breath in sequence had `tve_tvi_ratio < .25` OR first breath had `0.25 <= tve_tvi_ratio < 0.5` and `tve < 100`

Knowing this which features can we use here?

In [20]:
model = RandomForestClassifier()

# pick features based on expert selection. left for reader to determine best columns
columns_to_use = []
train_set = train_x[columns_to_use]
test_set = test_x[columns_to_use]

#model.fit(train_set, train_y_vector)
#predictions = model.predict(test_set)
#print(classification_report(test_y_vector, predictions))

### Other Methods

You are welcome to use other methods / mathematical functions for feature selection as well. I will briefly outline some of them here.


#### Wrapper Methods

This performs feature selection by brute force. Using your validation set, train many models with every single possible feature combination you can have. Determination of which features work best can be chosen based on the best performing model. Then you can apply this model to your testing set to determine performance.

Pros:
 * easy to understand
 * easy to code

Cons:
 * prone to overfitting
 * is time consuming. Must train $n!$ models if $n$ is the number of features.

#### PCA (Principal Component Analysis)

This method utilizes the [principal component analysis algorithm](https://en.wikipedia.org/wiki/Principal_component_analysis) to transform your dataset and generate new features that are independent of each other. The user gets to choose the number of features that are generated, and often modelers choose to generate an increasing number of features, and then train a new model for each PCA run while determining the performance of each model.

Pros:
 * Dimensionality reduction will cause models to train faster
 * Generated features are linearly uncorrelated with each other
 * Easy to utilize because there are multiple existing functions for this, like in [sckit-learn.](https://scikit-learn.org/stable/modules/generated/sklearn.decomposition.PCA.html)

Cons:
 * Loss of information in your data will likely occur and may cause performance degradation
 * Human comprehension of features is lost when PCA is performed

#### Mutual Information

[Mutual Information](https://en.wikipedia.org/wiki/Mutual_information) is similar to $\chi^2$ feature selection and measures the dependency between two variables. For machine learning this dependency can be measured between a feature and the target. It is equal to zero if and only if two random variables are independent, and higher values mean higher dependency.

Pros:
 * Fast
 * Supported by [Scikit-Learn](https://scikit-learn.org/stable/modules/generated/sklearn.feature_selection.mutual_info_classif.html#r50b872b699c4-1)

Cons:
 * Like $\chi^2$ may not generalize to the test set.

#### Mixed Methods

It is possible use a variety of methods in combination with each other. Generally expert feature selection is the first method used and then additional synthetic methods are added on top of this. Ultimately as the modeler, this work is on you to figure out how to perform best. One method might work for one problem and then completely fail for another. This is why it is often best to utilize as many possible methods as possible when performing modeling and only declare a winner when all other possible methods have been explored. It is critical to always beware of overfitting. Have a good validation set to evaluate your model, and don't pick your best methods versus your testing set. This is almost guaranteed to lead to overfitting.

### Finish Expert Feature Selection & Find another Feature Selection Method to Use.

Finish the coding for expert feature selection and use another feature selection method like PCA/mutual information/wrapper methods for use in your model. Which one performs best?

In [21]:
from sklearn.ensemble import RandomForestClassifier
from sklearn.preprocessing import MinMaxScaler
from sklearn.metrics import classification_report
from sklearn.feature_selection import SelectKBest, chi2, mutual_info_classif
from sklearn.decomposition import PCA
from sklearn.feature_selection import RFE
from sklearn.model_selection import train_test_split

# Expert Feature Selection
columns_to_use = ['tve_tvi_ratio', 'e_time', 'tve']

# Model with Expert Features
expert_train_set = train_x[columns_to_use]
expert_test_set = test_x[columns_to_use]

expert_model = RandomForestClassifier()
expert_model.fit(expert_train_set, train_y_vector)
expert_predictions = expert_model.predict(expert_test_set)

print("Classification Report for Expert Features:")
print(classification_report(test_y_vector, expert_predictions))

# Other Feature Selection Methods
# PCA
pca = PCA(n_components=2)
pca_train_set = pca.fit_transform(train_x)
pca_test_set = pca.transform(test_x)

pca_model = RandomForestClassifier()
pca_model.fit(pca_train_set, train_y_vector)
pca_predictions = pca_model.predict(pca_test_set)

print("Classification Report for PCA Features:")
print(classification_report(test_y_vector, pca_predictions))

# Mutual Information
mi_selector = SelectKBest(score_func=mutual_info_classif, k=2)
mi_train_set = mi_selector.fit_transform(train_x, train_y_vector)
mi_test_set = mi_selector.transform(test_x)

mi_model = RandomForestClassifier()
mi_model.fit(mi_train_set, train_y_vector)
mi_predictions = mi_model.predict(mi_test_set)

print("Classification Report for Mutual Information Features:")
print(classification_report(test_y_vector, mi_predictions))

# Wrapper Methods
rfe_selector = RFE(estimator=RandomForestClassifier(), n_features_to_select=2)
rfe_train_set = rfe_selector.fit_transform(train_x, train_y_vector)
rfe_test_set = rfe_selector.transform(test_x)

rfe_model = RandomForestClassifier()
rfe_model.fit(rfe_train_set, train_y_vector)
rfe_predictions = rfe_model.predict(rfe_test_set)

# Print classification report for the Wrapper Methods model
print("Classification Report for Wrapper Methods Features:")
print(classification_report(test_y_vector, rfe_predictions))

Classification Report for Expert Features:
              precision    recall  f1-score   support

           0       0.92      0.96      0.94       842
           1       0.81      0.98      0.89       301
           2       0.67      0.02      0.04       104

    accuracy                           0.89      1247
   macro avg       0.80      0.65      0.62      1247
weighted avg       0.87      0.89      0.85      1247

Classification Report for PCA Features:
              precision    recall  f1-score   support

           0       0.69      0.80      0.74       842
           1       0.37      0.29      0.32       301
           2       0.00      0.00      0.00       104

    accuracy                           0.61      1247
   macro avg       0.35      0.37      0.36      1247
weighted avg       0.55      0.61      0.58      1247

Classification Report for Mutual Information Features:
              precision    recall  f1-score   support

           0       0.90      0.95      0.93  

## Other Ways to Improve Your Model

### Class Imbalance

Class imbalance occurs when one class comprises a larger ratio of the observations in the dataset than another. This can be seen very clearly in our current training dataset.

In [22]:
train_y_vector.value_counts() / len(train_y_vector) * 100

0    72.38781
1    23.64367
2     3.96852
Name: count, dtype: float64

We can see here that normal observations comprise 72.3% of our training dataset, BSA is 19.68%, and DTA is 8.01%. This imbalance can have implications on the training of machine learning models because our model may not have enough information to learn effective class boundaries. Some algorithms are more resistant to class imbalance than others. Neural networks however are particularly affected by imbalance issues because of the nature of the way training is performed with these algorithms. Often algorithms besides neural networks benefit from techniques to reduce the class imbalance issue too. There are a number of techniques to tackle class imbalance.

#### ROS (Random Over-Sampling)

Random over-sampling aims to oversample minority classes by choosing observations at random with replacement until we  meet a certain ratio of majority to minority class observations. This is a fairly easy thing to code yourself if you wanted to do it, but just for ease we're going to use the [imbalanced-learn python package.](https://imbalanced-learn.readthedocs.io/en/stable/)

In [23]:
import imblearn

# get all columns in our dataset except patient and breath_id
columns_to_use = list(set(train_x.columns).difference(['patient', 'breath_id']))
# Initialize the RandomOverSampler. This initialization will give us 1:1:1 class ratios. If we want different
# ratios then we can chance the sampling_strategy input argument. For more details see the documentation
# https://imbalanced-learn.readthedocs.io/en/stable/generated/imblearn.over_sampling.RandomOverSampler.html#imblearn.over_sampling.RandomOverSampler
ros = imblearn.over_sampling.RandomOverSampler()
# re-sample the train set ONLY. Don't resample the testing set because otherwise you would be biasing model conclusions
train_x_ros, train_y_ros = ros.fit_resample(train_x[columns_to_use], train_y_vector)

# put the target vector into a series so we can just do some convenience function.
train_y_ros = pd.Series(train_y_ros)
# You'll see the dataset is equilibrated now with equal observations normal, BSA, and DTA breaths.
train_y_ros.value_counts()

# Now we can put this back into our model and see if performance changes. This is left for the reader

0    4323
1    4323
2    4323
Name: count, dtype: int64

#### RUS (Random Under-Sampling)

![](over-sampling-undersampling.png)

Random under-sampling is basically the inverse of the over-sampling technique. Instead of selecting with replacement from minority classes, here we randomly sample from the majority classes only until they meet some class ratio with the minority classes.

In [24]:
import imblearn

# get all columns in our dataset except patient and breath_id
columns_to_use = list(set(train_x.columns).difference(['patient', 'breath_id']))
# Initialize the RandomUnderSampler. This initialization will give us 1:1:1 class ratios. If we want different
# ratios then we can chance the sampling_strategy input argument. For more details see the documentation
# https://imbalanced-learn.readthedocs.io/en/stable/generated/imblearn.under_sampling.RandomUnderSampler.html#imblearn.under_sampling.RandomUnderSampler
rus = imblearn.under_sampling.RandomUnderSampler()
# re-sample the train set ONLY. Don't resample the testing set because otherwise you would be biasing model conclusions
train_x_rus, train_y_rus = rus.fit_resample(train_x[columns_to_use], train_y_vector)

# put the target vector into a series so we can just do some convenience function.
train_y_rus = pd.Series(train_y_rus)
# You'll see the dataset is equilibrated now with equal observations normal, BSA, and DTA breaths.
train_y_rus.value_counts()

# Now we can put this back into our model and see if performance changes. This is left for the reader

0    237
1    237
2    237
Name: count, dtype: int64

There are some downsides to RUS in that we are discarding data from the majority class which might be useful for the future. Also if some classes have very low ratios of data relative to the majority class then RUS may have more limited utility. With RUS, as with ROS, we will need to evaluate the effect of different class ratios on our validation set. Maybe a `4:2:1` ratio would be best for this problem, we just don't know until we try. I will leave this as an additional exercise for the reader.

#### SMOTE (Synthetic Minority Oversampling TEchnique)

One downside about the methods mentioned is that they always are drawn from the existing distribution of class data. It is quite possible that if we collected additional samples that there would be new observations that fit in between these existing data points. This is the intuition behind smote that can also be seen in the below image.

![](smote-intuition.png)

The benefit of SMOTE is that we are expanding our dataset, which means more data for our model to train on, while we are semi-intelligently generating new samples. Of course generated data may have no basis for reality, so good modeling habit should always check to see whether RUS, ROS, or SMOTE works best for a problem, and which class ratios work best for which technique.

In [25]:
import imblearn

# get all columns in our dataset except patient and breath_id
columns_to_use = list(set(train_x.columns).difference(['patient', 'breath_id']))
# Initialize SMOTE. This initialization will give us 1:1:1 class ratios. If we want different
# ratios then we can chance the sampling_strategy input argument. For more details see the documentation
# https://imbalanced-learn.readthedocs.io/en/stable/generated/imblearn.over_sampling.SMOTE.html#imblearn.over_sampling.SMOTE
smote = imblearn.over_sampling.SMOTE()
# re-sample the train set ONLY. Don't resample the testing set because otherwise you would be biasing model conclusions
train_x_smote, train_y_smote = smote.fit_resample(train_x[columns_to_use], train_y_vector)

# put the target vector into a series so we can just do some convenience function.
train_y_smote = pd.Series(train_y_smote)
# You'll see the dataset is equilibrated now with equal observations normal, BSA, and DTA breaths.
train_y_smote.value_counts()
# Now we can put this back into our model and see if performance changes. This is left for the reader

0    4323
1    4323
2    4323
Name: count, dtype: int64

## Assignment \#2 Utilize all 3 Imbalance Correction Techniques

Utilize ROS, RUS, and SMOTE with the following imbalance ratios: 0.3, 0.4, 0.5, 0.6, 0.7, 0.8, 0.9, 1.0. Is there an algorithm that performs best? Are there ratios of imbalance that perform best?

In [26]:
RF_model = RandomForestClassifier()

#### Imbalance Ratio = 0.3

In [27]:
ir = 0.3

In [28]:
n = 5000

ros = imblearn.over_sampling.RandomOverSampler(sampling_strategy = {0: n, 1: int(n*ir), 2: int(n*ir*ir)})

train_x_ros, train_y_ros = ros.fit_resample(train_x[columns_to_use], train_y_vector)

train_y_ros = pd.Series(train_y_ros)

train_y_ros.value_counts()

RF_model.fit(train_x_ros, train_y_ros)
predictions = RF_model.predict(test_x[columns_to_use])

for idx, pred in enumerate(predictions):
    if pred == 2:
        predictions[idx - 1] = 2

print(f"Imbalance Ratio: {ir} for ROS")
RF_report = classification_report(test_y_vector, predictions)
print(RF_report)


Imbalance Ratio: 0.3 for ROS
              precision    recall  f1-score   support

           0       0.90      0.96      0.93       842
           1       0.85      0.97      0.90       301
           2       1.00      0.02      0.04       104

    accuracy                           0.88      1247
   macro avg       0.91      0.65      0.62      1247
weighted avg       0.89      0.88      0.85      1247



In [29]:
n = 237

rus = imblearn.under_sampling.RandomUnderSampler(sampling_strategy={0: int(n*ir*ir), 1: int(n*ir), 2: n})

train_x_rus, train_y_rus = rus.fit_resample(train_x[columns_to_use], train_y_vector)

train_y_rus = pd.Series(train_y_rus)

train_y_rus.value_counts()

RF_model.fit(train_x_rus, train_y_rus)
predictions = RF_model.predict(test_x[columns_to_use])

for idx, pred in enumerate(predictions):
    if pred == 2:
        predictions[idx - 1] = 2

print(f"Imbalance Ratio: {ir} for RUS")
RF_report = classification_report(test_y_vector, predictions)
print(RF_report)

Imbalance Ratio: 0.3 for RUS
              precision    recall  f1-score   support

           0       1.00      0.37      0.54       842
           1       0.88      0.52      0.66       301
           2       0.13      0.97      0.23       104

    accuracy                           0.46      1247
   macro avg       0.67      0.62      0.48      1247
weighted avg       0.90      0.46      0.54      1247



In [30]:
n = 5000

smote = imblearn.over_sampling.SMOTE(sampling_strategy={0: n, 1: int(n*ir), 2: int(n*ir*ir)})

train_x_smote, train_y_smote = smote.fit_resample(train_x[columns_to_use], train_y_vector)

train_y_smote = pd.Series(train_y_smote)

train_y_smote.value_counts()

RF_model.fit(train_x_smote, train_y_smote)
predictions = RF_model.predict(test_x[columns_to_use])

for idx, pred in enumerate(predictions):
    if pred == 2:
        predictions[idx - 1] = 2

print(f"Imbalance Ratio: {ir} for SMOTE")
RF_report = classification_report(test_y_vector, predictions)
print(RF_report)


Imbalance Ratio: 0.3 for SMOTE
              precision    recall  f1-score   support

           0       0.90      0.96      0.93       842
           1       0.85      0.97      0.91       301
           2       1.00      0.06      0.11       104

    accuracy                           0.89      1247
   macro avg       0.92      0.66      0.65      1247
weighted avg       0.90      0.89      0.86      1247



#### Imbalance Ratio = 0.4

In [31]:
ir = 0.4

In [32]:
n = 5000

ros = imblearn.over_sampling.RandomOverSampler(sampling_strategy = {0: n, 1: int(n*ir), 2: int(n*ir*ir)})

train_x_ros, train_y_ros = ros.fit_resample(train_x[columns_to_use], train_y_vector)

train_y_ros = pd.Series(train_y_ros)

train_y_ros.value_counts()

RF_model.fit(train_x_ros, train_y_ros)
predictions = RF_model.predict(test_x[columns_to_use])

for idx, pred in enumerate(predictions):
    if pred == 2:
        predictions[idx - 1] = 2

print(f"Imbalance Ratio: {ir} for ROS")
RF_report = classification_report(test_y_vector, predictions)
print(RF_report)


Imbalance Ratio: 0.4 for ROS
              precision    recall  f1-score   support

           0       0.90      0.96      0.93       842
           1       0.85      0.97      0.90       301
           2       1.00      0.04      0.07       104

    accuracy                           0.89      1247
   macro avg       0.92      0.66      0.64      1247
weighted avg       0.90      0.89      0.85      1247



In [33]:
n = 237

rus = imblearn.under_sampling.RandomUnderSampler(sampling_strategy={0: int(n*ir*ir), 1: int(n*ir), 2: n})

train_x_rus, train_y_rus = rus.fit_resample(train_x[columns_to_use], train_y_vector)

train_y_rus = pd.Series(train_y_rus)

train_y_rus.value_counts()

RF_model.fit(train_x_rus, train_y_rus)
predictions = RF_model.predict(test_x[columns_to_use])

for idx, pred in enumerate(predictions):
    if pred == 2:
        predictions[idx - 1] = 2

print(f"Imbalance Ratio: {ir} for RUS")
RF_report = classification_report(test_y_vector, predictions)
print(RF_report)

Imbalance Ratio: 0.4 for RUS
              precision    recall  f1-score   support

           0       1.00      0.63      0.78       842
           1       0.80      0.69      0.74       301
           2       0.21      0.90      0.34       104

    accuracy                           0.67      1247
   macro avg       0.67      0.74      0.62      1247
weighted avg       0.89      0.67      0.73      1247



In [34]:
n = 5000

smote = imblearn.over_sampling.SMOTE(sampling_strategy={0: n, 1: int(n*ir), 2: int(n*ir*ir)})

train_x_smote, train_y_smote = smote.fit_resample(train_x[columns_to_use], train_y_vector)

train_y_smote = pd.Series(train_y_smote)

train_y_smote.value_counts()

RF_model.fit(train_x_smote, train_y_smote)
predictions = RF_model.predict(test_x[columns_to_use])

for idx, pred in enumerate(predictions):
    if pred == 2:
        predictions[idx - 1] = 2

print(f"Imbalance Ratio: {ir} for SMOTE")
RF_report = classification_report(test_y_vector, predictions)
print(RF_report)

Imbalance Ratio: 0.4 for SMOTE
              precision    recall  f1-score   support

           0       0.91      0.96      0.93       842
           1       0.86      0.97      0.91       301
           2       0.80      0.15      0.26       104

    accuracy                           0.89      1247
   macro avg       0.86      0.69      0.70      1247
weighted avg       0.89      0.89      0.87      1247



#### Imbalance Ratio = 0.5

In [35]:
ir = 0.5

In [36]:
n = 5000

ros = imblearn.over_sampling.RandomOverSampler(sampling_strategy = {0: n, 1: int(n*ir), 2: int(n*ir*ir)})

train_x_ros, train_y_ros = ros.fit_resample(train_x[columns_to_use], train_y_vector)

train_y_ros = pd.Series(train_y_ros)

train_y_ros.value_counts()

RF_model.fit(train_x_ros, train_y_ros)
predictions = RF_model.predict(test_x[columns_to_use])

for idx, pred in enumerate(predictions):
    if pred == 2:
        predictions[idx - 1] = 2

print(f"Imbalance Ratio: {ir} for ROS")
RF_report = classification_report(test_y_vector, predictions)
print(RF_report)

Imbalance Ratio: 0.5 for ROS
              precision    recall  f1-score   support

           0       0.91      0.96      0.93       842
           1       0.86      0.96      0.91       301
           2       0.67      0.12      0.20       104

    accuracy                           0.89      1247
   macro avg       0.81      0.68      0.68      1247
weighted avg       0.87      0.89      0.87      1247



In [37]:
n = 237

rus = imblearn.under_sampling.RandomUnderSampler(sampling_strategy={0: int(n*ir*ir), 1: int(n*ir), 2: n})

train_x_rus, train_y_rus = rus.fit_resample(train_x[columns_to_use], train_y_vector)

train_y_rus = pd.Series(train_y_rus)

train_y_rus.value_counts()

RF_model.fit(train_x_rus, train_y_rus)
predictions = RF_model.predict(test_x[columns_to_use])

for idx, pred in enumerate(predictions):
    if pred == 2:
        predictions[idx - 1] = 2

print(f"Imbalance Ratio: {ir} for RUS")
RF_report = classification_report(test_y_vector, predictions)
print(RF_report)

Imbalance Ratio: 0.5 for RUS
              precision    recall  f1-score   support

           0       0.98      0.62      0.75       842
           1       0.82      0.73      0.77       301
           2       0.20      0.84      0.32       104

    accuracy                           0.66      1247
   macro avg       0.66      0.73      0.61      1247
weighted avg       0.87      0.66      0.72      1247



In [38]:
n = 5000

smote = imblearn.over_sampling.SMOTE(sampling_strategy={0: n, 1: int(n*ir), 2: int(n*ir*ir)})

train_x_smote, train_y_smote = smote.fit_resample(train_x[columns_to_use], train_y_vector)

train_y_smote = pd.Series(train_y_smote)

train_y_smote.value_counts()

RF_model.fit(train_x_smote, train_y_smote)
predictions = RF_model.predict(test_x[columns_to_use])

for idx, pred in enumerate(predictions):
    if pred == 2:
        predictions[idx - 1] = 2

print(f"Imbalance Ratio: {ir} for SMOTE")
RF_report = classification_report(test_y_vector, predictions)
print(RF_report)

Imbalance Ratio: 0.5 for SMOTE
              precision    recall  f1-score   support

           0       0.91      0.96      0.93       842
           1       0.87      0.96      0.91       301
           2       0.73      0.15      0.25       104

    accuracy                           0.89      1247
   macro avg       0.83      0.69      0.70      1247
weighted avg       0.88      0.89      0.87      1247



#### Imbalance Ratio = 0.6

In [39]:
ir = 0.6

In [40]:
n = 5000

ros = imblearn.over_sampling.RandomOverSampler(sampling_strategy = {0: n, 1: int(n*ir), 2: int(n*ir*ir)})

train_x_ros, train_y_ros = ros.fit_resample(train_x[columns_to_use], train_y_vector)

train_y_ros = pd.Series(train_y_ros)

train_y_ros.value_counts()

RF_model.fit(train_x_ros, train_y_ros)
predictions = RF_model.predict(test_x[columns_to_use])

for idx, pred in enumerate(predictions):
    if pred == 2:
        predictions[idx - 1] = 2

print(f"Imbalance Ratio: {ir} for ROS")
RF_report = classification_report(test_y_vector, predictions)
print(RF_report)

Imbalance Ratio: 0.6 for ROS
              precision    recall  f1-score   support

           0       0.90      0.96      0.93       842
           1       0.85      0.96      0.90       301
           2       0.67      0.04      0.07       104

    accuracy                           0.89      1247
   macro avg       0.81      0.66      0.64      1247
weighted avg       0.87      0.89      0.85      1247



In [41]:
n = 237

rus = imblearn.under_sampling.RandomUnderSampler(sampling_strategy={0: int(n*ir*ir), 1: int(n*ir), 2: n})

train_x_rus, train_y_rus = rus.fit_resample(train_x[columns_to_use], train_y_vector)

train_y_rus = pd.Series(train_y_rus)

train_y_rus.value_counts()

RF_model.fit(train_x_rus, train_y_rus)
predictions = RF_model.predict(test_x[columns_to_use])

for idx, pred in enumerate(predictions):
    if pred == 2:
        predictions[idx - 1] = 2

print(f"Imbalance Ratio: {ir} for RUS")
RF_report = classification_report(test_y_vector, predictions)
print(RF_report)

Imbalance Ratio: 0.6 for RUS
              precision    recall  f1-score   support

           0       0.99      0.77      0.87       842
           1       0.75      0.81      0.78       301
           2       0.28      0.73      0.40       104

    accuracy                           0.77      1247
   macro avg       0.68      0.77      0.68      1247
weighted avg       0.88      0.77      0.81      1247



In [42]:
n = 5000

smote = imblearn.over_sampling.SMOTE(sampling_strategy={0: n, 1: int(n*ir), 2: int(n*ir*ir)})

train_x_smote, train_y_smote = smote.fit_resample(train_x[columns_to_use], train_y_vector)

train_y_smote = pd.Series(train_y_smote)

train_y_smote.value_counts()

RF_model.fit(train_x_smote, train_y_smote)
predictions = RF_model.predict(test_x[columns_to_use])

for idx, pred in enumerate(predictions):
    if pred == 2:
        predictions[idx - 1] = 2

print(f"Imbalance Ratio: {ir} for SMOTE")
RF_report = classification_report(test_y_vector, predictions)
print(RF_report)

Imbalance Ratio: 0.6 for SMOTE
              precision    recall  f1-score   support

           0       0.91      0.95      0.93       842
           1       0.85      0.95      0.90       301
           2       0.44      0.12      0.18       104

    accuracy                           0.88      1247
   macro avg       0.73      0.67      0.67      1247
weighted avg       0.85      0.88      0.86      1247



#### Imbalance Ratio = 0.7

In [43]:
ir = 0.7

In [44]:
n = 5000

ros = imblearn.over_sampling.RandomOverSampler(sampling_strategy = {0: n, 1: int(n*ir), 2: int(n*ir*ir)})

train_x_ros, train_y_ros = ros.fit_resample(train_x[columns_to_use], train_y_vector)

train_y_ros = pd.Series(train_y_ros)

train_y_ros.value_counts()

RF_model.fit(train_x_ros, train_y_ros)
predictions = RF_model.predict(test_x[columns_to_use])

for idx, pred in enumerate(predictions):
    if pred == 2:
        predictions[idx - 1] = 2

print(f"Imbalance Ratio: {ir} for ROS")
RF_report = classification_report(test_y_vector, predictions)
print(RF_report)

Imbalance Ratio: 0.7 for ROS
              precision    recall  f1-score   support

           0       0.90      0.96      0.93       842
           1       0.84      0.97      0.90       301
           2       1.00      0.04      0.07       104

    accuracy                           0.88      1247
   macro avg       0.91      0.65      0.63      1247
weighted avg       0.89      0.88      0.85      1247



In [45]:
n = 237

rus = imblearn.under_sampling.RandomUnderSampler(sampling_strategy={0: int(n*ir*ir), 1: int(n*ir), 2: n})

train_x_rus, train_y_rus = rus.fit_resample(train_x[columns_to_use], train_y_vector)

train_y_rus = pd.Series(train_y_rus)

train_y_rus.value_counts()

RF_model.fit(train_x_rus, train_y_rus)
predictions = RF_model.predict(test_x[columns_to_use])

for idx, pred in enumerate(predictions):
    if pred == 2:
        predictions[idx - 1] = 2

print(f"Imbalance Ratio: {ir} for RUS")
RF_report = classification_report(test_y_vector, predictions)
print(RF_report)

Imbalance Ratio: 0.7 for RUS
              precision    recall  f1-score   support

           0       0.97      0.81      0.88       842
           1       0.76      0.83      0.79       301
           2       0.24      0.51      0.32       104

    accuracy                           0.79      1247
   macro avg       0.66      0.71      0.67      1247
weighted avg       0.86      0.79      0.81      1247



In [46]:
n = 5000

smote = imblearn.over_sampling.SMOTE(sampling_strategy={0: n, 1: int(n*ir), 2: int(n*ir*ir)})

train_x_smote, train_y_smote = smote.fit_resample(train_x[columns_to_use], train_y_vector)

train_y_smote = pd.Series(train_y_smote)

train_y_smote.value_counts()

RF_model.fit(train_x_smote, train_y_smote)
predictions = RF_model.predict(test_x[columns_to_use])

for idx, pred in enumerate(predictions):
    if pred == 2:
        predictions[idx - 1] = 2

print(f"Imbalance Ratio: {ir} for SMOTE")
RF_report = classification_report(test_y_vector, predictions)
print(RF_report)

Imbalance Ratio: 0.7 for SMOTE
              precision    recall  f1-score   support

           0       0.92      0.93      0.92       842
           1       0.87      0.93      0.90       301
           2       0.48      0.31      0.38       104

    accuracy                           0.88      1247
   macro avg       0.76      0.72      0.73      1247
weighted avg       0.87      0.88      0.87      1247



#### Imbalance Ratio = 0.8

In [47]:
ir = 0.8

In [48]:
n = 5000

ros = imblearn.over_sampling.RandomOverSampler(sampling_strategy = {0: n, 1: int(n*ir), 2: int(n*ir*ir)})

train_x_ros, train_y_ros = ros.fit_resample(train_x[columns_to_use], train_y_vector)

train_y_ros = pd.Series(train_y_ros)

train_y_ros.value_counts()

RF_model.fit(train_x_ros, train_y_ros)
predictions = RF_model.predict(test_x[columns_to_use])

for idx, pred in enumerate(predictions):
    if pred == 2:
        predictions[idx - 1] = 2

print(f"Imbalance Ratio: {ir} for ROS")
RF_report = classification_report(test_y_vector, predictions)
print(RF_report)

Imbalance Ratio: 0.8 for ROS
              precision    recall  f1-score   support

           0       0.90      0.96      0.93       842
           1       0.85      0.97      0.91       301
           2       1.00      0.04      0.07       104

    accuracy                           0.89      1247
   macro avg       0.92      0.66      0.64      1247
weighted avg       0.90      0.89      0.85      1247



In [49]:
n = 237

rus = imblearn.under_sampling.RandomUnderSampler(sampling_strategy={0: int(n*ir*ir), 1: int(n*ir), 2: n})

train_x_rus, train_y_rus = rus.fit_resample(train_x[columns_to_use], train_y_vector)

train_y_rus = pd.Series(train_y_rus)

train_y_rus.value_counts()

RF_model.fit(train_x_rus, train_y_rus)
predictions = RF_model.predict(test_x[columns_to_use])

for idx, pred in enumerate(predictions):
    if pred == 2:
        predictions[idx - 1] = 2

print(f"Imbalance Ratio: {ir} for RUS")
RF_report = classification_report(test_y_vector, predictions)
print(RF_report)

Imbalance Ratio: 0.8 for RUS
              precision    recall  f1-score   support

           0       0.99      0.64      0.78       842
           1       0.80      0.79      0.79       301
           2       0.20      0.76      0.31       104

    accuracy                           0.69      1247
   macro avg       0.66      0.73      0.63      1247
weighted avg       0.88      0.69      0.74      1247



In [50]:
n = 5000

smote = imblearn.over_sampling.SMOTE(sampling_strategy={0: n, 1: int(n*ir), 2: int(n*ir*ir)})

train_x_smote, train_y_smote = smote.fit_resample(train_x[columns_to_use], train_y_vector)

train_y_smote = pd.Series(train_y_smote)

train_y_smote.value_counts()

RF_model.fit(train_x_smote, train_y_smote)
predictions = RF_model.predict(test_x[columns_to_use])

for idx, pred in enumerate(predictions):
    if pred == 2:
        predictions[idx - 1] = 2

print(f"Imbalance Ratio: {ir} for SMOTE")
RF_report = classification_report(test_y_vector, predictions)
print(RF_report)

Imbalance Ratio: 0.8 for SMOTE
              precision    recall  f1-score   support

           0       0.91      0.93      0.92       842
           1       0.87      0.93      0.90       301
           2       0.42      0.24      0.30       104

    accuracy                           0.87      1247
   macro avg       0.73      0.70      0.71      1247
weighted avg       0.86      0.87      0.86      1247



#### Imbalance Ratio = 0.9

In [51]:
ir = 0.9

In [52]:
n = 5000

ros = imblearn.over_sampling.RandomOverSampler(sampling_strategy = {0: n, 1: int(n*ir), 2: int(n*ir*ir)})

train_x_ros, train_y_ros = ros.fit_resample(train_x[columns_to_use], train_y_vector)

train_y_ros = pd.Series(train_y_ros)

train_y_ros.value_counts()

RF_model.fit(train_x_ros, train_y_ros)
predictions = RF_model.predict(test_x[columns_to_use])

for idx, pred in enumerate(predictions):
    if pred == 2:
        predictions[idx - 1] = 2

print(f"Imbalance Ratio: {ir} for ROS")
RF_report = classification_report(test_y_vector, predictions)
print(RF_report)

Imbalance Ratio: 0.9 for ROS
              precision    recall  f1-score   support

           0       0.90      0.96      0.93       842
           1       0.85      0.97      0.90       301
           2       1.00      0.04      0.07       104

    accuracy                           0.89      1247
   macro avg       0.92      0.66      0.64      1247
weighted avg       0.90      0.89      0.85      1247



In [53]:
n = 237

rus = imblearn.under_sampling.RandomUnderSampler(sampling_strategy={0: int(n*ir*ir), 1: int(n*ir), 2: n})

train_x_rus, train_y_rus = rus.fit_resample(train_x[columns_to_use], train_y_vector)

train_y_rus = pd.Series(train_y_rus)

train_y_rus.value_counts()

RF_model.fit(train_x_rus, train_y_rus)
predictions = RF_model.predict(test_x[columns_to_use])

for idx, pred in enumerate(predictions):
    if pred == 2:
        predictions[idx - 1] = 2

print(f"Imbalance Ratio: {ir} for RUS")
RF_report = classification_report(test_y_vector, predictions)
print(RF_report)

Imbalance Ratio: 0.9 for RUS
              precision    recall  f1-score   support

           0       0.97      0.86      0.91       842
           1       0.77      0.88      0.82       301
           2       0.27      0.41      0.33       104

    accuracy                           0.83      1247
   macro avg       0.67      0.72      0.69      1247
weighted avg       0.86      0.83      0.84      1247



In [54]:
n = 5000

smote = imblearn.over_sampling.SMOTE(sampling_strategy={0: n, 1: int(n*ir), 2: int(n*ir*ir)})

train_x_smote, train_y_smote = smote.fit_resample(train_x[columns_to_use], train_y_vector)

train_y_smote = pd.Series(train_y_smote)

train_y_smote.value_counts()

RF_model.fit(train_x_smote, train_y_smote)
predictions = RF_model.predict(test_x[columns_to_use])

for idx, pred in enumerate(predictions):
    if pred == 2:
        predictions[idx - 1] = 2

print(f"Imbalance Ratio: {ir} for SMOTE")
RF_report = classification_report(test_y_vector, predictions)
print(RF_report)

Imbalance Ratio: 0.9 for SMOTE
              precision    recall  f1-score   support

           0       0.92      0.93      0.92       842
           1       0.88      0.94      0.91       301
           2       0.49      0.33      0.39       104

    accuracy                           0.88      1247
   macro avg       0.76      0.73      0.74      1247
weighted avg       0.87      0.88      0.87      1247



#### Imbalance Ratio = 1.0

In [55]:
ir = 1

In [56]:
n = 5000

ros = imblearn.over_sampling.RandomOverSampler(sampling_strategy = {0: n, 1: int(n*ir), 2: int(n*ir*ir)})

train_x_ros, train_y_ros = ros.fit_resample(train_x[columns_to_use], train_y_vector)

train_y_ros = pd.Series(train_y_ros)

train_y_ros.value_counts()

RF_model.fit(train_x_ros, train_y_ros)
predictions = RF_model.predict(test_x[columns_to_use])

for idx, pred in enumerate(predictions):
    if pred == 2:
        predictions[idx - 1] = 2

print(f"Imbalance Ratio: {ir} for ROS")
RF_report = classification_report(test_y_vector, predictions)
print(RF_report)

Imbalance Ratio: 1 for ROS
              precision    recall  f1-score   support

           0       0.90      0.97      0.93       842
           1       0.85      0.97      0.91       301
           2       1.00      0.04      0.07       104

    accuracy                           0.89      1247
   macro avg       0.92      0.66      0.64      1247
weighted avg       0.90      0.89      0.86      1247



In [57]:
n = 237

rus = imblearn.under_sampling.RandomUnderSampler(sampling_strategy={0: int(n*ir*ir), 1: int(n*ir), 2: n})

train_x_rus, train_y_rus = rus.fit_resample(train_x[columns_to_use], train_y_vector)

train_y_rus = pd.Series(train_y_rus)

train_y_rus.value_counts()

RF_model.fit(train_x_rus, train_y_rus)
predictions = RF_model.predict(test_x[columns_to_use])

for idx, pred in enumerate(predictions):
    if pred == 2:
        predictions[idx - 1] = 2

print(f"Imbalance Ratio: {ir} for RUS")
RF_report = classification_report(test_y_vector, predictions)
print(RF_report)

Imbalance Ratio: 1 for RUS
              precision    recall  f1-score   support

           0       0.96      0.86      0.91       842
           1       0.77      0.87      0.82       301
           2       0.31      0.45      0.37       104

    accuracy                           0.83      1247
   macro avg       0.68      0.73      0.70      1247
weighted avg       0.86      0.83      0.84      1247



In [58]:
n = 5000

smote = imblearn.over_sampling.SMOTE(sampling_strategy={0: n, 1: int(n*ir), 2: int(n*ir*ir)})

train_x_smote, train_y_smote = smote.fit_resample(train_x[columns_to_use], train_y_vector)

train_y_smote = pd.Series(train_y_smote)

train_y_smote.value_counts()

RF_model.fit(train_x_smote, train_y_smote)
predictions = RF_model.predict(test_x[columns_to_use])

for idx, pred in enumerate(predictions):
    if pred == 2:
        predictions[idx - 1] = 2

print(f"Imbalance Ratio: {ir} for SMOTE")
RF_report = classification_report(test_y_vector, predictions)
print(RF_report)

Imbalance Ratio: 1 for SMOTE
              precision    recall  f1-score   support

           0       0.92      0.89      0.90       842
           1       0.87      0.90      0.88       301
           2       0.30      0.36      0.33       104

    accuracy                           0.85      1247
   macro avg       0.70      0.71      0.70      1247
weighted avg       0.86      0.85      0.85      1247



#### Using a for loop to generate the results again

In [59]:
from sklearn.metrics import accuracy_score

imbalance_ratios = [0.3, 0.4, 0.5, 0.6, 0.7, 0.8, 0.9, 1.0]

for ir in imbalance_ratios:
    # Random Over-sampling (ROS)
    n = 5000
    ros = imblearn.over_sampling.RandomOverSampler(sampling_strategy={0: n, 1: int(n * ir), 2: int(n * ir * ir)})
    train_x_ros, train_y_ros = ros.fit_resample(train_x[columns_to_use], train_y_vector)
    train_y_ros = pd.Series(train_y_ros)
    RF_model.fit(train_x_ros, train_y_ros)
    predictions = RF_model.predict(test_x[columns_to_use])
    for idx, pred in enumerate(predictions):
        if pred == 2:
            predictions[idx - 1] = 2
    print(f"Imbalance Ratio: {ir} for ROS")
    RF_report = classification_report(test_y_vector, predictions)
    print(RF_report)

    # Random Under-sampling (RUS)
    n = 237
    rus = imblearn.under_sampling.RandomUnderSampler(sampling_strategy={0: int(n * ir * ir), 1: int(n * ir), 2: n})
    train_x_rus, train_y_rus = rus.fit_resample(train_x[columns_to_use], train_y_vector)
    train_y_rus = pd.Series(train_y_rus)
    RF_model.fit(train_x_rus, train_y_rus)
    predictions = RF_model.predict(test_x[columns_to_use])
    for idx, pred in enumerate(predictions):
        if pred == 2:
            predictions[idx - 1] = 2
    print(f"Imbalance Ratio: {ir} for RUS")
    RF_report = classification_report(test_y_vector, predictions)
    print(RF_report)

    # SMOTE
    n = 5000
    smote = imblearn.over_sampling.SMOTE(sampling_strategy={0: n, 1: int(n * ir), 2: int(n * ir * ir)})
    train_x_smote, train_y_smote = smote.fit_resample(train_x[columns_to_use], train_y_vector)
    train_y_smote = pd.Series(train_y_smote)
    RF_model.fit(train_x_smote, train_y_smote)
    predictions = RF_model.predict(test_x[columns_to_use])
    for idx, pred in enumerate(predictions):
        if pred == 2:
            predictions[idx - 1] = 2
    print(f"Imbalance Ratio: {ir} for SMOTE")
    RF_report = classification_report(test_y_vector, predictions)
    print(RF_report)

Imbalance Ratio: 0.3 for ROS
              precision    recall  f1-score   support

           0       0.90      0.96      0.93       842
           1       0.85      0.97      0.90       301
           2       1.00      0.02      0.04       104

    accuracy                           0.89      1247
   macro avg       0.92      0.65      0.62      1247
weighted avg       0.90      0.89      0.85      1247

Imbalance Ratio: 0.3 for RUS
              precision    recall  f1-score   support

           0       1.00      0.07      0.13       842
           1       0.83      0.54      0.65       301
           2       0.09      0.89      0.17       104

    accuracy                           0.25      1247
   macro avg       0.64      0.50      0.32      1247
weighted avg       0.88      0.25      0.26      1247

Imbalance Ratio: 0.3 for SMOTE
              precision    recall  f1-score   support

           0       0.90      0.96      0.93       842
           1       0.86      0.96      0

#### Interpretation of the Results

Based on the results of the two tests, comparing the f1-score for each class, the performance increases with imbalance rate increasing, and SMOTE and RUS with imbalance rate = 1.0 work best. SMOTE at an imbalance ratio of 1.0 generates synthetic samples for balance, and RUS at the same ratio achieves balance by reducing the majority class. Both methods aim to provide the model with a balanced dataset, potentially leading to enhanced classification performance, especially when the imbalance ratio is significant or the majority class contains irrelevant or noisy data.

With the oversampling ratio {0: n, 1: int(n * ir), 2: int(n * ir * ir)}, we are increasing the minority class samples as the imbalance ratio (ir) increases. This can help in making the dataset more balanced, allowing the model to better capture the minority class. On the other hand, the undersampling strategy {0: int(n * ir * ir), 1: int(n * ir), 2: n} indeed retains a portion of the majority class as the imbalance ratio (ir) increases. This allows the model to maintain some balance between classes while focusing more on the minority class through oversampling. This balance can help the model generalize better.

##### SMOTE (Synthetic Minority Over-sampling Technique) at an Imbalance Ratio of 1.0:

SMOTE generates synthetic examples for the minority class by interpolating between existing minority class samples. These new samples are created to bridge the gap between the minority and majority classes. When SMOTE is applied with an imbalance ratio of 1.0, it means that the number of synthetic samples in the minority class is increased to match the number of samples in the majority class. This results in a balanced class distribution. The interpolation process of SMOTE ensures that the synthetic examples are within the feature space, making them representative of the minority class. This balance facilitates the model's learning process and reduces the dominance of the majority class, enhancing the model's capacity to accurately classify instances from the minority class.

##### Random Under-Sampling (RUS) at an Imbalance Ratio of 1.0:

In contrast to SMOTE, RUS reduces the number of samples from both the majority and minority classes to achieve balance.When RUS is applied with an imbalance ratio of 1.0, it signifies that the majority class is reduced to match the number of samples in the minority class. This, too, results in a balanced class distribution.While decreasing the majority class may seem counterintuitive, in practice, it can mitigate the dominance of the majority class and prevent the model from overfitting to that class. By attaining a balanced dataset through RUS, the model is allowed to focus more on the minority class. This approach can be effective, particularly when the majority class contains noisy or redundant data that could impede the model's ability to correctly classify the minority class.


## How to Systematically Evaluate Your Model

### k-fold cross-validation

K-fold cross-validation is a resampling procedure used to systematically evaluate machine learning models. The procedure has a parameter called k that refers to the number of groups that a given data sample is to be split into. This technique is important in machine learning because it ensures that every observation from the original dataset has the chance of appearing in the training and test set, providing a thorough assessment of how well a model performs across different subsets of data.

In [60]:
from sklearn.model_selection import StratifiedKFold

##Define k
k = 5

##initialize k fold setup for cross-validation
kf = StratifiedKFold(n_splits=k, shuffle=True, random_state=42)

In [ ]:
#Remember your train_set_scaled must be already defined from Pre-Lab!

##Choose your model
model = RandomForestClassifier()

for train_index, val_index in kf.split(train_set_scaled, train_y_vector):

      X_train, X_val = train_set_scaled[train_index], train_set_scaled[val_index]
      y_train, y_val = train_y_vector[train_index], train_y_vector[val_index]

      # Now we can fit the model, generate predictions, and compare the performance with validation labels y_val.
      # Once that is done, we save the results for next fold/iteration

## Assignment \#3 Perform k-fold cross-validation

Evaluate an ML model of your choosing (RF, Logistic Regression, etc...) under 5-fold cross-validation. Print the macro average precision, recall, and f1-scores in each fold. Print the average values across all 5 folds at the end of the loop.

In [62]:
# XXX code here
import numpy as np
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import precision_score, recall_score, f1_score
from sklearn.model_selection import StratifiedKFold

macro_precision_scores = []
macro_recall_scores = []
macro_f1_scores = []
i = 0
k = 5

columns_to_use = list(set(train_x.columns).difference(['patient', 'breath_id']))

scaler = StandardScaler()
# Fit the scaler to training data, and then scale the training data.
scaler.fit(train_x[columns_to_use])
train_x_s = scaler.transform(train_x[columns_to_use])

##Choose your model
RF_model = RandomForestClassifier()

kf = StratifiedKFold(n_splits=k, shuffle=True, random_state=42)

for train_index, val_index in kf.split(train_x_s, train_y_vector):

      X_train, X_val = train_x_s[train_index], train_x_s[val_index]
      y_train, y_val = train_y_vector[train_index], train_y_vector[val_index]

      # Now we can fit the model, generate predictions, and compare the performance with validation labels y_val.
      # Once that is done, we save the results for next fold/iteration
      RF_model.fit(X_train, y_train)
      predictions = RF_model.predict(X_val)
      
      for idx, pred in enumerate(predictions):
            if pred == 2:
                  predictions[idx - 1] = 2
      
      i = i + 1
      
      precision = precision_score(y_val, predictions, average='macro')
      recall = recall_score(y_val, predictions, average='macro')
      f1 = f1_score(y_val, predictions, average='macro')
      
      macro_precision_scores.append(precision)
      macro_recall_scores.append(recall)
      macro_f1_scores.append(f1)
      
      print(f"Fold {i} - Precision: {precision:.4f}, Recall: {recall:.4f}, F1-Score: {f1:.4f}")
      print(classification_report(y_val, predictions))
      
avg_precision = np.mean(macro_precision_scores)
avg_recall = np.mean(macro_recall_scores)
avg_f1 = np.mean(macro_f1_scores)
print(f"Average Precision: {avg_precision:.4f}, Average Recall: {avg_recall:.4f}, Average F1-Score: {avg_f1:.4f}")

Fold 1 - Precision: 0.8264, Recall: 0.8715, F1-Score: 0.8455
              precision    recall  f1-score   support

           0       0.98      0.97      0.98       865
           1       0.95      0.94      0.94       282
           2       0.55      0.71      0.62        48

    accuracy                           0.95      1195
   macro avg       0.83      0.87      0.85      1195
weighted avg       0.96      0.95      0.95      1195

Fold 2 - Precision: 0.8071, Recall: 0.8787, F1-Score: 0.8323
              precision    recall  f1-score   support

           0       0.98      0.96      0.97       865
           1       0.97      0.92      0.94       282
           2       0.47      0.75      0.58        48

    accuracy                           0.95      1195
   macro avg       0.81      0.88      0.83      1195
weighted avg       0.96      0.95      0.95      1195

Fold 3 - Precision: 0.7926, Recall: 0.8695, F1-Score: 0.8168
              precision    recall  f1-score   support



In the tutorial, you are taught to use the function StratifiedKFold. What does it mean when we stratify our k-folds?

It means that the data is divided into k-folds while ensuring that each fold maintains the same class distribution as the original dataset,making the proportion of each class in the dataset is preserved in each fold. 

Stratified k-fold is particularly useful when dealing with imbalanced datasets, where some classes have significantly fewer samples than others. Without stratification, there's a risk that a fold could end up with a very imbalanced class distribution, which could lead to biased model evaluation. By using stratified k-fold cross-validation, we help ensure that the model is trained and evaluated on representative subsets of the data.